In [2]:
from keras.preprocessing.image import img_to_array, load_img
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
import numpy as np
import os
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
import matplotlib as mlp
import matplotlib.pyplot as plt
from matplotlib.image import imread
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model
from PIL import Image
# label_index = {'landfill_trash': 6, 'plastic': 5, 'paper': 4, 'mental': 3, 'glass': 2, 'cardboard': 1};


In [12]:
model_vgg = VGG16(weights='imagenet', include_top=False)

In [3]:
def modelProcess(img_path, model):
    img = load_img(img_path, target_size=(224, 224)) 
    img = img_to_array(img) 
    x = np.expand_dims(img, axis=0) 
    x = preprocess_input(x) 
    x_vgg = model.predict(x) 
    
    x_vgg = x_vgg.reshape(1, 25088)
    return x_vgg

In [4]:
def transform_format(path):  
    folders = os.listdir(path)  
    for j in range(len(folders)):
        dirName = path + '//' + folders[j] + '//' 
        li = os.listdir(dirName)  
        for filename in li:
            newname = filename
            newname = newname.split(".")  
            if newname[-1] != "png":  
                newname[-1] = "png"
                newname = str.join(".", newname)  
                filename = dirName + filename
                newname = dirName + newname
                os.rename(filename, newname)  
                print('reading the images:%s' % (newname))  
                a = np.array(Image.open(newname))  
                if ((len(a.shape) != 3) or (a.shape[2] != 3)):  
                    a = np.array(Image.open(newname).convert('RGB'))  
                    img = Image.fromarray(a.astype('uint8'))  
                    img.save(newname)  
                    print(a.shape)  

In [5]:
def read_data(path):
    folders = os.listdir(path) 
    for j in range(len(folders)): 
        folder = path + '//' + folders[j] 
        dirs = os.listdir(folder) 
       
        img_path = []
        for i in dirs:
            if os.path.splitext(i)[1] == ".png": 
                img_path.append(i)
        img_path = [folder + "//" + i for i in img_path] 
        features1 = np.zeros([len(img_path), 25088]) 
        for i in range(len(img_path)):
            feature_i = modelProcess(img_path[i], model_vgg) 
            # print('preprocessed:', img_path[i])
            features1[i] = feature_i
        if j == 0: 
            X = features1 
        else:
            X = np.concatenate((X, features1), axis=0)
           
    return X 
       

In [10]:
def read_label(path):
    y = []
    folders = os.listdir(path)  
    for j in range(len(folders)): 
        dirName = path + '//' + folders[j] + '//' 
        lens = len(os.listdir(dirName)) 
        for i in range(lens):
            y.append(j) 
    lb = LabelBinarizer() 
    y = lb.fit_transform(y)
    return y

In [11]:
path = '/Users/sheny/Desktop/PROJECT/Module3/Train' 

y = []
folders = os.listdir(path) 
for j in range(len(folders)): 
    dirName = path + '//' + folders[j] + '//'  
    lens = len(os.listdir(dirName)) 
    for i in range(lens):
        y.append(j) 

lb = LabelBinarizer()
y = lb.fit_transform(y) 

transform_format(path)  
X = read_data(path) 


NameError: name 'model_vgg' is not defined

In [ ]:
print(folders)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.18, random_state=50)
print('X_train.shape:', X_train.shape)
print('X_test.shape:', X_test.shape)
print('y_train.shape:', y_train.shape)
print('y_test.shape:', y_test.shape)

In [ ]:
# from keras.layers.core import Dropout
from tensorflow.python.keras import layers

In [ ]:
model = Sequential()
model.add(Dense(units=40, activation='relu', input_dim=25088))
# model.add(BatchNormalization())
model.add(Dense(units=6, activation='softmax'))

model.summary()


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=39)

In [ ]:
y_train_predict = model.predict_classes(X_train) 
y_train = lb.inverse_transform(y_train)

accuracy_train = recall_score(y_train, y_train_predict, average='macro') 
print('-' * 35)
print('accuracy_train:', accuracy_train)


y_test_predict = model.predict_classes(X_test)
y_test = lb.inverse_transform(y_test)


accuracy_test = recall_score(y_test, y_test_predict, average='macro')

print('-' * 35)
print('accuracy_test:', accuracy_test)

In [ ]:
print(X_test)
print(y_test_predict)

In [ ]:
test_path = 'C:/Users/sheny/Desktop/PROJECT/Module3/te1/te/'

transform_format('/'.join(test_path.split('/')[:-2]))

# folders = os.listdir(test_path)  
# num = len(folders)
label_index = {'cardboard': 1, 'glass': 2, 'mental': 3, 'paper': 4, 'plastic': 5,'landfill_trash': 6}


In [ ]:
print(len(folders))
new_test_path ='C:/Users/sheny/Desktop/Module3/te1/'
z = read_data(new_test_path)
result = model.predict_classes(z)
# final_result =[]
len(result)

In [ ]:
from pandas import *
idx = Int64Index(range(1264,2528))

data_file = {'Id':idx,'Prediction':result
    
}
df = DataFrame(data_file,columns =['Id','Prediction'])


In [ ]:
# df['Prediction'] =df['Prediction'].replace(0,"cardboard")
replace_value ={0:'cardboard',1:'glass',2:'landfill_trash'}
df = df.replace({"Prediction":replace_value})
replace_value2 ={'cardboard':1,'glass':2,'landfill_trash':6}
df = df.replace({"Prediction":replace_value2})
# df.replace({'Prediction':{'cardboard':1}},{'Prediction':{'glass':2}},{'Prediction':{'landfill_trash':6}})

In [ ]:
df.to_csv('kaggle_vgg_2.csv',header=True,index=False)